In [1]:
!pip install transformers --quiet
!pip install datasets --quiet
!pip install seqeval --quiet

In [2]:
import pandas as pd
# import torch 
import numpy as np
import tensorflow as tf
from tensorflow import keras
from transformers import BertTokenizerFast, BertForTokenClassification, TFBertForTokenClassification, TFBertModel
# from torch.utils.data import DataLoader
from tqdm import tqdm
# from torch.optim import SGD
from datasets import list_datasets, load_dataset, load_metric, Dataset, DatasetDict, ClassLabel, Sequence
from sklearn.metrics import classification_report

2023-04-01 17:22:27.498547: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-01 17:22:27.587205: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-01 17:22:27.587219: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-01 17:22:28.064820: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [3]:
ds_relation = load_dataset('ade_corpus_v2', 'Ade_corpus_v2_drug_ade_relation', split='train')
ds_relation

Found cached dataset ade_corpus_v2 (/home/diana/.cache/huggingface/datasets/ade_corpus_v2/Ade_corpus_v2_drug_ade_relation/1.0.0/940d61334dbfac6b01ac5d00286a2122608b8dc79706ee7e9206a1edb172c559)


Dataset({
    features: ['text', 'drug', 'effect', 'indexes'],
    num_rows: 6821
})

In [4]:
df = ds_relation.to_pandas()
df.head()

,text,drug,effect,indexes
0,Intravenous azithromycin-induced ototoxicity.,azithromycin,ototoxicity,"{'drug': {'start_char': [12], 'end_char': [24]..."
1,"Immobilization, while Paget's bone disease was...",dihydrotachysterol,increased calcium-release,"{'drug': {'start_char': [91], 'end_char': [109..."
2,Unaccountable severe hypercalcemia in a patien...,dihydrotachysterol,hypercalcemia,"{'drug': {'start_char': [84], 'end_char': [102..."
3,METHODS: We report two cases of pseudoporphyri...,naproxen,pseudoporphyria,"{'drug': {'start_char': [58], 'end_char': [66]..."
4,METHODS: We report two cases of pseudoporphyri...,oxaprozin,pseudoporphyria,"{'drug': {'start_char': [71], 'end_char': [80]..."


In [5]:
df_relation = df['indexes'].apply(pd.Series)
df_relation[['drug_start_char', 'drug_end_char']] = df_relation.apply(lambda row: pd.Series(row['drug']), axis=1)
df_relation = df_relation.drop('drug', axis=1)
df_relation[['effect_start_char', 'effect_end_char']] = df_relation.apply(lambda row: pd.Series(row['effect']), axis=1)
df_relation = df_relation.drop('effect', axis=1)
df = pd.concat([df, df_relation], axis=1)
df = df.drop('indexes', axis=1)
df.head()

,text,drug,effect,drug_start_char,drug_end_char,effect_start_char,effect_end_char
0,Intravenous azithromycin-induced ototoxicity.,azithromycin,ototoxicity,[12],[24],[33],[44]
1,"Immobilization, while Paget's bone disease was...",dihydrotachysterol,increased calcium-release,[91],[109],[143],[168]
2,Unaccountable severe hypercalcemia in a patien...,dihydrotachysterol,hypercalcemia,[84],[102],[21],[34]
3,METHODS: We report two cases of pseudoporphyri...,naproxen,pseudoporphyria,[58],[66],[32],[47]
4,METHODS: We report two cases of pseudoporphyri...,oxaprozin,pseudoporphyria,[71],[80],[32],[47]


In [6]:
# save to JSON to then import into Dataset object
df.to_json("dataset.json", orient="records", lines=True)

In [7]:
ds_ade_relation = load_dataset("json", data_files="dataset.json", split='train')
ds_ade_relation

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /home/diana/.cache/huggingface/datasets/json/default-ba6a86503022fdee/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


Dataset({
    features: ['text', 'drug', 'effect', 'drug_start_char', 'drug_end_char', 'effect_start_char', 'effect_end_char'],
    num_rows: 6821
})

In [8]:
label_list = ['O', 'B-DRUG', 'I-DRUG', 'B-EFFECT', 'I-EFFECT']
custom_seq = Sequence(feature=ClassLabel(num_classes=5, 
                                         names=label_list,
                                         names_file=None, id=None), length=-1, id=None)

ds_ade_relation.features["ner_tags"] = custom_seq
# ds_relation_tts['test'].features["ner_tags"] = custom_seq

In [9]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

In [10]:
model = TFBertModel.from_pretrained('bert-base-cased', num_labels=len(label_list))

2023-04-01 17:22:37.765439: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-04-01 17:22:37.765467: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-01 17:22:37.765480: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (diana-MS-7D19): /proc/driver/nvidia/version does not exist
2023-04-01 17:22:37.765633: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Some layers from the model checkpoint at bert-base-cased were not used

In [11]:
# t1 = tokenizer(ds_relation['text'][0], padding='max_length', max_length=25, truncation=True)
# t1

In [12]:
# t1_word_id = t1.word_ids()
# t1_word_id

In [13]:
MAX_SEQUENCE_LENGTH = 119

In [14]:
def generate_row_labels(row, verbose=False):
    """ Given a row from the consolidated `Ade_corpus_v2_drug_ade_relation` dataset, 
    generate BIO tags for drug and effect entities. 
    
    """

    text = row["text"]

    labels = []
    label = "O"
    prefix = ""
    bio_label = []
    
    # while iterating through tokens, increment to traverse all drug and effect spans
    drug_index = 0
    effect_index = 0
    
    tokens = tokenizer(text, return_offsets_mapping=True, truncation=True, padding='max_length', max_length=MAX_SEQUENCE_LENGTH)

    for n in range(len(tokens["input_ids"])):
        offset_start, offset_end = tokens["offset_mapping"][n]

        # should only happen for [CLS] and [SEP]
        if offset_end - offset_start == 0:
            labels.append(0) #-100
            continue
        
        if drug_index < len(row["drug_start_char"]) and offset_start == row["drug_start_char"][drug_index]:
            label = "DRUG"
            prefix = "B-"

        elif effect_index < len(row["effect_start_char"]) and offset_start == row["effect_start_char"][effect_index]:
            label = "EFFECT"
            prefix = "B-"
        
        labels.append(label_list.index(f"{prefix}{label}"))
        bio_label.append(f"{prefix}{label}")
            
        if drug_index < len(row["drug_end_char"]) and offset_end == row["drug_end_char"][drug_index]:
            label = "O"
            prefix = ""
            drug_index += 1
            
        elif effect_index < len(row["effect_end_char"]) and offset_end == row["effect_end_char"][effect_index]:
            label = "O"
            prefix = ""
            effect_index += 1

        # need to transition "inside" if we just entered an entity
        if prefix == "B-":
            prefix = "I-"
    
    if verbose:
        print(f"{row}\n")
        orig = tokenizer.convert_ids_to_tokens(tokens["input_ids"])
        for n in range(len(labels)):
            print(orig[n], labels[n])
    tokens["labels"] = labels
    tokens['bio_label'] = bio_label
    
    return tokens

In [15]:
labeled_dataset = ds_ade_relation.map(generate_row_labels)
labeled_dataset

Map:   0%|          | 0/6821 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'drug', 'effect', 'drug_start_char', 'drug_end_char', 'effect_start_char', 'effect_end_char', 'input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'labels', 'bio_label'],
    num_rows: 6821
})

In [16]:
df = labeled_dataset.to_pandas()
df.head()

,text,drug,effect,drug_start_char,drug_end_char,effect_start_char,effect_end_char,input_ids,token_type_ids,attention_mask,offset_mapping,labels,bio_label
0,Intravenous azithromycin-induced ototoxicity.,azithromycin,ototoxicity,[12],[24],[33],[44],"[101, 1130, 4487, 7912, 2285, 170, 5303, 1582,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[[0, 0], [0, 2], [2, 5], [5, 8], [8, 11], [12,...","[0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 0, 0, 3, 4, ...","[O, O, O, O, B-DRUG, I-DRUG, I-DRUG, I-DRUG, I..."
1,"Immobilization, while Paget's bone disease was...",dihydrotachysterol,increased calcium-release,[91],[109],[143],[168],"[101, 146, 6262, 12809, 15100, 117, 1229, 8500...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[[0, 0], [0, 1], [1, 3], [3, 5], [5, 14], [14,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,Unaccountable severe hypercalcemia in a patien...,dihydrotachysterol,hypercalcemia,[84],[102],[21],[34],"[101, 24258, 14566, 19529, 2165, 5199, 177, 24...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[[0, 0], [0, 3], [3, 6], [6, 10], [10, 13], [1...","[0, 0, 0, 0, 0, 0, 3, 4, 4, 4, 4, 0, 0, 0, 0, ...","[O, O, O, O, O, B-EFFECT, I-EFFECT, I-EFFECT, ..."
3,METHODS: We report two cases of pseudoporphyri...,naproxen,pseudoporphyria,[58],[66],[32],[47],"[101, 22157, 24162, 15609, 1708, 131, 1284, 25...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[[0, 0], [0, 2], [2, 4], [4, 6], [6, 7], [7, 8...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 4, ...","[O, O, O, O, O, O, O, O, O, O, B-EFFECT, I-EFF..."
4,METHODS: We report two cases of pseudoporphyri...,oxaprozin,pseudoporphyria,[71],[80],[32],[47],"[101, 22157, 24162, 15609, 1708, 131, 1284, 25...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[[0, 0], [0, 2], [2, 4], [4, 6], [6, 7], [7, 8...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 4, ...","[O, O, O, O, O, O, O, O, O, O, B-EFFECT, I-EFF..."


In [17]:
len(df)

6821

In [18]:
label = [l for i in df['bio_label'] for l in i]

unique_labels = set()
for lb in label:
    [unique_labels.add(lb)]# for i in lb if i not in unique_labels]
unique_labels

{'B-DRUG', 'B-EFFECT', 'I-DRUG', 'I-EFFECT', 'O'}

In [19]:
# labels_to_ids = {k: v for v, k in enumerate(unique_labels)}
# ids_to_labels = {v: k for v, k in enumerate(unique_labels)}
labels_to_ids = {'O': 0, 'B-DRUG': 1, 'I-DRUG': 2, 'B-EFFECT': 3, 'I-EFFECT': 4}
labels_to_ids

{'O': 0, 'B-DRUG': 1, 'I-DRUG': 2, 'B-EFFECT': 3, 'I-EFFECT': 4}

In [20]:
label_all_tokens = False

def align_label(texts, labels):
  tokenized_inputs = tokenizer(texts, padding='max_length', max_length=MAX_SEQUENCE_LENGTH, truncation=True)
  word_ids = tokenized_inputs.word_ids()

  l = []
  previous = None

  for idx, i in enumerate(word_ids):
      if i is None:
          l.append(0) #-100
      elif i != previous:
          try:
              l.append(labels_to_ids[labels[idx-1]])
          except:
              l.append(0) #-100
      else:
          try:
              l.append(labels_to_ids[labels[idx-1]]) # if label_all_tokens else -100)
          except:
              l.append(0) #-100
      previous = i

  return l

In [21]:
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42),
                            [int(.8 * len(df)), int(.9 * len(df))])

In [22]:
# Train Data
train_lbl = [list(i) for i in df_train['bio_label']]
train_txt =  df_train['text'].values.tolist()
train_encodings = tokenizer(train_txt, #str(train_idx),
                                padding=True, 
                                max_length = MAX_SEQUENCE_LENGTH, 
                                truncation=True, 
                                return_tensors="tf") #for train_idx in train_txt
# train_labels = [align_label(i,j) for i,j in zip(train_txt, train_lbl)]
train_labels = list(df_train['labels'])
# train_labels = tf.convert_to_tensor(np.array(list(df_train['labels'])))

# Validation Data
val_lbl = [list(i) for i in df_val['bio_label']]
val_txt =  df_val['text'].values.tolist()
val_encodings = tokenizer(val_txt,#str(val_idx),
                                padding='max_length', 
                                max_length = MAX_SEQUENCE_LENGTH, 
                                truncation=True, 
                                return_tensors="tf") #for val_idx in val_txt]
# val_labels = [align_label(i,j) for i,j in zip(val_txt, val_lbl)]
val_labels = list(df_val['labels'])

# Test Data
test_lbl = [list(i) for i in df_test['bio_label']]
test_txt =  df_test['text'].values.tolist()
test_encodings = tokenizer(test_txt,#str(test_idx),
                                padding='max_length', 
                                max_length = MAX_SEQUENCE_LENGTH, 
                                truncation=True, 
                                return_tensors="tf") #for test_idx in test_txt]
# test_labels = [align_label(i,j) for i,j in zip(test_txt, test_lbl)]
test_labels = list(df_test['labels'])

In [23]:
# train_txt

In [24]:
# train_labels

In [25]:
# train_encodings.input_ids[:1]

In [26]:
# train_encodings.token_type_ids[:1]

In [27]:
# train_labels[:4]

In [28]:
# for i,j in zip(train_txt, train_lbl):
#     print(align_label(i,j))

In [29]:
# print(train_lbl[5])
# print(train_labels[5])
# print(train_txt[5])
# print(tokenizer.tokenize(train_txt[5]))
# print(train_encodings.input_ids[5])
# print(tokenizer(train_txt[5]).word_ids())

In [30]:
# bert_input = tokenizer(train_txt[:4], 
#               max_length=119,
#               truncation=True,
#               padding='max_length', 
#               return_tensors='tf')

# bert_input

In [31]:
# len(model.weights)

In [32]:
# model.weights[0]

In [33]:
# bert_output = model(bert_input)
# bert_output[0]

In [34]:
# bert_output[0].shape

In [35]:
# bert_output[1].shape

In [36]:
# train_encodings

In [37]:
def create_bert_cls_model(max_sequence_length=MAX_SEQUENCE_LENGTH,
                          hidden_size = 100, 
                          num_classes = 5,
                          dropout=0.05,
                          learning_rate=0.00001):

  model.trainable=True    

  input_ids = tf.keras.layers.Input(shape=(max_sequence_length,), dtype=tf.int64, name='input_ids_layer')
  token_type_ids = tf.keras.layers.Input(shape=(max_sequence_length,), dtype=tf.int64, name='token_type_ids_layer')
  attention_mask = tf.keras.layers.Input(shape=(max_sequence_length,), dtype=tf.int64, name='attention_mask_layer')

  bert_inputs = {'input_ids': input_ids,
                 'token_type_ids': token_type_ids,
                 'attention_mask': attention_mask}

  bert_out = model(bert_inputs)

  cls_token = bert_out[0]

  last_hidden_output = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer')(cls_token)
  last_hidden_output = tf.keras.layers.Dropout(dropout)(last_hidden_output)  

  classification = tf.keras.layers.Dense(num_classes, activation='softmax',name='classification_layer')(last_hidden_output)
  
  classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=[classification])
  
  classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                               metrics='sparse_categorical_accuracy')

  
  return classification_model

In [38]:
bert_model = create_bert_cls_model(num_classes=len(label_list))

In [39]:
bert_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 attention_mask_layer (InputLay  [(None, 119)]       0           []                               
 er)                                                                                              
                                                                                                  
 input_ids_layer (InputLayer)   [(None, 119)]        0           []                               
                                                                                                  
 token_type_ids_layer (InputLay  [(None, 119)]       0           []                               
 er)                                                                                              
                                                                                              

In [ ]:
bert_model_history = bert_model.fit([train_encodings.input_ids[:200], train_encodings.token_type_ids[:200], train_encodings.attention_mask[:200]], 
                                    np.array(train_labels[:200]),   
                                    validation_data=([val_encodings.input_ids[:200], val_encodings.token_type_ids[:200], val_encodings.attention_mask[:200]], 
                                    np.array(val_labels[:200])),    
                                    batch_size=16, 
                                    epochs=5)

bert_model_history

Epoch 1/5
13/13 [==============================] - 93s 6s/step - loss: 0.7524 - sparse_categorical_accuracy: 0.7906 - val_loss: 0.3849 - val_sparse_categorical_accuracy: 0.9093
Epoch 2/5
13/13 [==============================] - 75s 6s/step - loss: 0.3214 - sparse_categorical_accuracy: 0.9150 - val_loss: 0.2565 - val_sparse_categorical_accuracy: 0.9105
Epoch 3/5
 4/13 [========>.....................] - ETA: 40s - loss: 0.2531 - sparse_categorical_accuracy: 0.9145

In [ ]:
bert_score = bert_model.evaluate([test_encodings.input_ids, test_encodings.token_type_ids, test_encodings.attention_mask], 
                                 np.array(test_labels))

print('Test loss', bert_score[0])
print('Test accuracy', bert_score[1])

In [ ]:
bert_predictions = bert_model.predict([test_encodings.input_ids[:], test_encodings.token_type_ids[:], test_encodings.attention_mask[:]])
bert_predictions = tf.argmax(bert_predictions, axis=-1)
bert_predictions

In [ ]:
bert_predictions[0]

In [ ]:
metric = load_metric("seqeval")
bert_predictions = bert_model.predict([test_encodings.input_ids[:], test_encodings.token_type_ids[:], test_encodings.attention_mask[:]])
predictions = np.argmax(bert_predictions, axis=-1)
labels = np.array(test_labels)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

In [ ]:
true_predictions

In [ ]:
true_labels

In [ ]:
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove special tokens
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
results = metric.compute(predictions=np.array(bert_predictions), references=np.array(test_labels))
results

In [ ]:
from sklearn.metrics import f1_score

for prediction, label in zip(np.array(bert_predictions), np.array(test_labels)):
    f1 = f1_score(prediction, label, average='micro')
    print(f1)


In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

# def precision_m(y_true, y_pred):
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
#     precision = true_positives / (predicted_positives + K.epsilon())
#     return precision

# def f1_m(y_true, y_pred):
#     precision = precision_m(y_true, y_pred)
#     recall = recall_m(y_true, y_pred)
#     return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
recall_m(np.array(bert_predictions), np.array(test_labels))

In [ ]:
tf.cast(bert_predictions, tf.int64)

In [ ]:
x = K.variable([1, 2, 3, 4, 5])

# Clip the tensor values to be between 2 and 4
x_clipped = K.clip(x, 2, 4)

x_clipped

In [ ]:
true_positives = K.sum(K.round(K.clip(tf.cast(bert_predictions, tf.int64) * test_labels, 0, 1)))
possible_positives = K.sum(K.round(K.clip(tf.cast(bert_predictions, tf.int64), 0, 1)))
true_positives / (possible_positives + K.epsilon())

In [ ]:
def create_bert_cls_model2(max_sequence_length=MAX_SEQUENCE_LENGTH,
                          hidden_size = 100, 
                          num_classes = 5,
                          dropout=0.05,
                          learning_rate=0.00001):

    input_ids = tf.keras.layers.Input(shape=(max_sequence_length,), dtype=tf.int64, name='input_ids_layer')
    token_type_ids = tf.keras.layers.Input(shape=(max_sequence_length,), dtype=tf.int64, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(max_sequence_length,), dtype=tf.int64, name='attention_mask_layer')

    bert_inputs = {'input_ids': input_ids,
                 'token_type_ids': token_type_ids,
                 'attention_mask': attention_mask}

    bert_out = model(bert_inputs)

    cls_token = bert_out[0]

    last_hidden_output = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer')(cls_token)
    last_hidden_output = tf.keras.layers.Dropout(dropout)(last_hidden_output)  

    classification = tf.keras.layers.Dense(num_classes, activation='softmax',name='classification_layer')(last_hidden_output)

    classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=[classification])

    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                               metrics=['sparse_categorical_accuracy', tf.keras.metrics.Recall()])


    return classification_model

In [ ]:
bert_model2 = create_bert_cls_model2(num_classes=len(label_list))

In [ ]:
bert_model2.summary()

In [ ]:
bert_model_history2 = bert_model2.fit([train_encodings.input_ids, train_encodings.token_type_ids, train_encodings.attention_mask], 
                                    np.array(train_labels),   
                                    validation_data=([val_encodings.input_ids, val_encodings.token_type_ids, val_encodings.attention_mask], 
                                    np.array(val_labels)),    
                                    batch_size=16, 
                                    epochs=5)

bert_model_history2